# Description
This script's purpose is to reproject the images.

Reprojecting the image involves changing the coordinate system of the images from one coordinate system to another .

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os

In [ ]:
#Defining the source and destination paths 
srcPath = '20200319/'
dstPath = 'data/reprojected/20200319/'

In [ ]:
#Obtaining a list of the available images 
lstRst = [band for band in os.listdir(srcPath) if band.endswith(".tif")]
lstRst
# len(lstRst)

In [ ]:
#Opening the images and viewing their coordinate system  
expRst = rasterio.open(srcPath + lstRst[0])
expRst.crs

In [ ]:
#Defining the coordinate system to convert to
dstCrs = {'init': 'EPSG:32631'}

In [ ]:
def reprojectRaster(srcRst, dstRst, dstCrs, srcPath, dstPath) :
    srcRst = rasterio.open(srcPath + srcRst)

    #calculate transform array and shape of reprojected raster
    transform, width, height = calculate_default_transform(
        srcRst.crs, dstCrs, srcRst.width, srcRst.height, *srcRst.bounds)
    
    # working of the meta for the destination raster
    kwargs = srcRst.meta.copy()
    kwargs.update({
        'crs': dstCrs,
        'transform': transform,
        'width': width,
        'height': height
        })
    
    #open destination raster
    dstRst = rasterio.open(dstPath + dstRst, 'w', **kwargs)
    
    #reproject and save raster band 
    for i in range (1, srcRst.count + 1):
        reproject(
            source=rasterio.band(srcRst, i),
            destination=rasterio.band(dstRst, i),
            #src_transform=srcRst.transform,
            src_crs=srcRst.crs,
            #dst_transform=transform,
            dst_crs=dstCrs,
            resampling=Resampling.nearest)
    
    #close destination raster
    dstRst.close()
    

In [ ]:
for srcRst in lstRst:
    dstRst = "reprojected_" + srcRst[0:] 
    reprojectRaster(srcRst, dstRst, dstCrs, srcPath, dstPath)
    print("Reprojection of %s done"%srcRst)
    